In [0]:
# Carga de los secretos
container_name = dbutils.secrets.get(scope="secretliga1", key="adlscontenedor")
adls_account_name = dbutils.secrets.get(scope="secretliga1", key="adlsstorage")
client_id = dbutils.secrets.get(scope="secretliga1", key="clientid")
client_secret = dbutils.secrets.get(scope="secretliga1", key="secretidt")
tenant_id = dbutils.secrets.get(scope="secretliga1", key="tenantidt")

# Configuración de Spark para la autenticación
spark.conf.set(f"fs.azure.account.auth.type.{adls_account_name}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{adls_account_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{adls_account_name}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{adls_account_name}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{adls_account_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token")

mount_point=f"/mnt/{container_name}"
try:    mounts = [m.mountPoint for m in dbutils.fs.mounts()]
except Exception as e:
    mounts = []


if mount_point in mounts:
  print(f"El montaje {mount_point} ya existe. Listando contenido:")
  display(dbutils.fs.ls(mount_point))
  return container_name, adls_account_name
else:
  print(f"Montando {mount_point}...")
  adls_direct_path = f"abfss://{container_name}@{adls_account_name}.dfs.core.windows.net/"
  print("Contenido del contendor usando acceso directo:")
  display(dbutils.fs.ls(adls_direct_path))